# Угадай число
Нужно написать программу, которая угадывает число за минимальное число попыток.

## Условия соревнования
- Компьютер загадывает целое число от 1 до 100, и нам его нужно угадать. Под «угадать», подразумевается «написать программу, которая угадывает число».    
- Алгоритм учитывает информацию о том, больше ли случайное число или меньше нужного нам.
- Необходимо добиться того, чтобы программа угадывала число меньше, чем за 20 попыток.


Импортируем библиотеку, которая нам пригодится для генерации случайных чисел. В следующих темах вы познакомитесь с ней подробнее:

In [1]:
import numpy as np

Ниже уже даны два простейших подхода к решению этой задачи. Посмотрим на них.

## Подход 1: Случайное угадывание

Простейший способ решения: научить программу случайным образом выбирать число до тех пор, пока оно не будет угадано. Этот способ не дает хорошего результата, однако будет для нас хорошей стартовой точкой.

In [2]:
def random_predict(number: int = 1) -> int:
    """Просто угадываем на random, никак не используя информацию о больше или меньше.
       Функция принимает загаданное число и возвращает число попыток

    Args:
        number (int, optional): Загаданное число. Defaults to 1.

    Returns:
        int: Число попыток
    """
    count = 0

    while True:
        count += 1
        predict_number = np.random.randint(1, 101)  # предполагаемое число
        if number == predict_number:
            break  # выход из цикла если угадали

    return count


## Подход 2: Угадывание с коррекцией

Сначала устанавливаем любое случайное число, а потом уменьшаем или увеличиваем его в зависимости от того, больше оно или меньше нужного.

In [3]:
def game_core_v2(number: int = 1) -> int:
    """Сначала устанавливаем любое random число, а потом уменьшаем
    или увеличиваем его в зависимости от того, больше оно или меньше нужного.
       Функция принимает загаданное число и возвращает число попыток

    Args:
        number (int, optional): Загаданное число. Defaults to 1.

    Returns:
        int: Число попыток
    """
    count = 0
    predict = np.random.randint(1, 101)

    while number != predict:
        count += 1
        if number > predict:
            predict += 1
        elif number < predict:
            predict -= 1

    return count

Посмотрим, как эти программы справляются с задачей.

## Функция для оценки

Эта функция необходима, чтобы определить, за какое число попыток программа угадывает наше число.

In [4]:
def score_game(random_predict) -> int:
    """За какое количество попыток в среднем за 1000 подходов угадывает наш алгоритм

    Args:
        random_predict ([type]): функция угадывания

    Returns:
        int: среднее количество попыток
    """
    count_ls = []
    # np.random.seed(1)  # фиксируем сид для воспроизводимости
    random_array = np.random.randint(1, 101, size=(1000))  # загадали список чисел

    for number in random_array:
        count_ls.append(random_predict(number))

    score = int(np.mean(count_ls))
    print(f'Ваш алгоритм угадывает число: в среднем за {score} попытки')
    print(f'Ваш алгоритм угадывает число: максимум за {max(count_ls)} попытки')
    print(f'Ваш алгоритм угадывает число: минимум за {min(count_ls)} попытки')

## Оценка работы алгоритмов
Определяем, какой подход лучше.

In [5]:
#Run benchmarking to score effectiveness of all algorithms
print('Run benchmarking for random_predict: ')
score_game(random_predict)

print('Run benchmarking for game_core_v2:')
score_game(game_core_v2)

Run benchmarking for random_predict: 
Ваш алгоритм угадывает число: в среднем за 99 попытки
Ваш алгоритм угадывает число: максимум за 701 попытки
Ваш алгоритм угадывает число: минимум за 1 попытки
Run benchmarking for game_core_v2:
Ваш алгоритм угадывает число: в среднем за 32 попытки
Ваш алгоритм угадывает число: максимум за 98 попытки
Ваш алгоритм угадывает число: минимум за 0 попытки


Как мы видим, две предложенные программы показывают не лучший результат.
Теперь вы попробуете решить эту задачу и найти более обтимальный алгоритм.

## Подход 3: Угадываем с использованием бинарного поиска

*Так как задан отсортированный (по возрастанию) список, мы можем использовать бинарный поиск.*
На первом шаге компьютер выбирает любое случайное число из диапазона (1, 101). Потом производим деление заданного списока чисел (1, 101) пополам и сравниваем средний элемент с загаданным числом. Повторяем процесс для той половины, в которой может находиться искомый элемент, и так до тех пор пока не будет достигнута цель.

In [6]:
def game_core_v3(number: int = 1) -> int:
    """
    Args:
        number (int, optional): Загаданное число. Defaults to 1.

    Returns:
        int: Число попыток
    """
    # задаем границы поиска
    first = 0 # начальный элемент
    last = 101 # конечныйй элемент
    count = 1 # задаём счётчик попыток (1 в случае нахождения загаданного числа копьютером сразу)
    predict_num = np.random.randint(1, 101) # генерация компьютером числа с которого начнем поиск

    while number != predict_num: # Зададим условие при котором цикл будет выполняться
        count += 1
        if predict_num > number: # если сгенерированное компьютером число больше загаданного
            last = predict_num # сдвигаем конечный элемент поиска до сгенерированного компьютером числа
            predict_num = (first + last) // 2 # теперь предполагаемое число это число из середины получившегося списка
        else: # если сгенерированное компьютером число меньше загаданного
            first = predict_num # сдвигаем начальный элемент поиска до сгенерированного компьютером числа
            predict_num = (first + last) // 2 # теперь предполагаемое число это число из середины получившегося списка

    return count # возвращаем полученый результат

Оценим качество вашего алгоритма:

In [7]:
print('Run benchmarking for game_core_v3:')
score_game(game_core_v3)

Run benchmarking for game_core_v3:
Ваш алгоритм угадывает число: в среднем за 6 попытки
Ваш алгоритм угадывает число: максимум за 8 попытки
Ваш алгоритм угадывает число: минимум за 1 попытки


Напомним, что для успешного решения задания программа должна угадывать число меньше чем за 20 попыток!